# Discovery Fuentes datos.gob.es

In [1]:

import pandas as pd
import requests

### Descarga del dataset

Se utilizará un dataset público proporcionado por el Instituto Nacional de Estadística de España disponible en datos.gob.es.

In [34]:
# URL del archivo CSV
URL = "https://www.ine.es/jaxiT3/files/t/csv_bdsc/48423.csv"

# Ruta específica donde se desea guardar el archivo CSV descargado
PATH = "data/tourism_spain_dataset.csv"

In [5]:
# Realizamos la solicitud HTTP para descargar el archivo
r = requests.get(URL)

# Verificamos que la solicitud fue exitosa (código de estado 200)
if r.status_code == 200:
    # Guardamos la información descargada en el archivo local
    with open(PATH, 'wb') as f:
        f.write(r.content)
    print(f"Archivo descargado exitosamente en: {PATH}")
else:
    print(f"Error al descargar el archivo. Código de estado: {r.status_code}")

Archivo descargado exitosamente en: data/tourism_spain_dataset.csv


### Carga del dataset

Cargamos nuestro dataset dentro de un dataframe de `pandas` que nos permite manipular, limpiar y visualizar fácilmente nuestros datos.

In [6]:
df = pd.read_csv(PATH, delimiter=';')

Utilizamos el método `data.info` para conocer el tipo de dato de cada columna y la cantidad de valores no nulos.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12420 entries, 0 to 12419
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Concepto turístico                12420 non-null  object
 1   Total Nacional                    12420 non-null  object
 2   Comunidades y Ciudades Autónomas  11799 non-null  object
 3   Residencia del viajero            12420 non-null  object
 4   Periodo                           12420 non-null  object
 5   Total                             11907 non-null  object
dtypes: object(6)
memory usage: 582.3+ KB


In [8]:
df.head()

,Concepto turístico,Total Nacional,Comunidades y Ciudades Autónomas,Residencia del viajero,Periodo,Total
0,Número de alojamientos turísticos ocupados,Total Nacional,NaN,Total,2023M09,1.019.000
1,Número de alojamientos turísticos ocupados,Total Nacional,NaN,Total,2023M08,1.287.000
2,Número de alojamientos turísticos ocupados,Total Nacional,NaN,Total,2023M07,1.164.000
3,Número de alojamientos turísticos ocupados,Total Nacional,NaN,Total,2023M06,775.000
4,Número de alojamientos turísticos ocupados,Total Nacional,NaN,Total,2023M05,691.000


In [9]:
df['Comunidades y Ciudades Autónomas'].unique()

array([nan, '01 Andalucía', '02 Aragón', '03 Asturias, Principado de',
       '04 Balears, Illes', '05 Canarias', '06 Cantabria',
       '07 Castilla y León', '08 Castilla - La Mancha', '09 Cataluña',
       '10 Comunitat Valenciana', '11 Extremadura', '12 Galicia',
       '13 Madrid, Comunidad de', '14 Murcia, Región de',
       '15 Navarra, Comunidad Foral de', '16 País Vasco', '17 Rioja, La',
       '18 Ceuta', '19 Melilla'], dtype=object)

In [19]:
df[['Comunidades y Ciudades Autónomas', 'Periodo', 'Concepto turístico', 'Total']].drop_duplicates().sort_values(by='Comunidades y Ciudades Autónomas')

,Comunidades y Ciudades Autónomas,Periodo,Concepto turístico,Total
207,01 Andalucía,2023M09,Número de alojamientos turísticos ocupados,NaN
8548,01 Andalucía,2018M08,Estancia media,"4,6"
8547,01 Andalucía,2018M09,Estancia media,"3,9"
8546,01 Andalucía,2018M10,Estancia media,"3,6"
8545,01 Andalucía,2018M11,Estancia media,"3,4"
...,...,...,...,...
8482,NaN,2018M05,Estancia media,"4,4"
8483,NaN,2018M04,Estancia media,"4,5"
8484,NaN,2018M03,Estancia media,"4,9"
8485,NaN,2018M02,Estancia media,"5,3"


In [33]:
df.sample(50).to_csv('test/tourism_spain_sample.csv')

In [31]:
df.query('`Comunidades y Ciudades Autónomas`== "01 Andalucía"').sample(50).to_csv('test/Andalucía_sample.csv')